In [1]:
import json
from selenium import webdriver
import os
import re
from bs4 import BeautifulSoup

# from selenium.webdriver.chrome.options import Options

# webdriver-manager
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# selenium tools
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webdriver import WebElement
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep
import json
from datetime import datetime

In [2]:
#selenium start
driver = webdriver.Chrome()

In [3]:
# Fazendo a requisição

post_video = "https://www.linkedin.com/posts/fernandogomesfg_python-gis-geospatial-ugcPost-7184308650679689216-ljBu?utm_source=share&utm_medium=member_desktop"
post_texto = "https://www.linkedin.com/posts/thiagolisboapro_99-das-pessoas-aqui-no-in-vivem-na-roda-activity-7186371514911383552-g_U9?utm_source=share&utm_medium=member_desktop"
post_imagem = "https://www.linkedin.com/posts/randypitcherii_snowflake-snowpark-where-have-you-been-activity-7186451080187195392-IXLk?utm_source=share&utm_medium=member_desktop"
post_mult_imagem = "https://www.linkedin.com/posts/rodrigobcerqueira_%C3%A9-poss%C3%ADvel-utilizar-dados-e-m%C3%A9todos-estat%C3%ADsticos-ugcPost-7181702612499447808-jwVv?utm_source=share&utm_medium=member_desktop"


url = """
https://www.linkedin.com/posts/marciana-agostinho_analytics-sql-analisededados-activity-7185970201480286208-Ynl6?utm_source=share&utm_medium=member_desktop
"""



driver.get(url)

In [4]:
def verify_sign_in():
    xpath_close_modal = "//icon[contains(@class, 'contextual-sign-in-modal__modal-dismiss-icon')]"
    try:
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, xpath_close_modal))
        ).click()
        return True
    except TimeoutException:
        return False
    
verify_sign_in()


True

In [5]:
def verify_content_unavailable():
    xpath_content_unavailable = "//a[@data-tracking-control-name='public_post_content-unavailable-join']"
    try:
        WebDriverWait(driver, 1.5).until(
            EC.element_to_be_clickable((By.XPATH, xpath_content_unavailable))
        )
        return True
    except TimeoutException:
        return False
    
if verify_content_unavailable():
    print("Conteúdo disponível apenas para usuários, parando o script!")

In [6]:
scrap = {}
for i in range(9):
    driver.execute_script(f"window.scrollTo(0, {300 * i});")
    sleep(.5)

sleep(.5)

#data-tracking-control-name="public_post_content-unavailable-join"


article_element = driver.find_element(by=By.TAG_NAME, value="article")
soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")

#Original Poster
xpath_post_header = "//div[@data-test-id='main-feed-activity-card__entity-lockup']"
header = soup_article.find("div", attrs={"data-test-id": "main-feed-activity-card__entity-lockup"})
header_content = [item.strip() for item in header.text.split("\n") if item.strip()]

scrap["autor_name"] = header.find(attrs={"data-tracking-control-name":"public_post_feed-actor-name"}).text.strip()
scrap["autor_subtitle"] = header_content[1]
scrap["post_age"] = header_content[2]

scrap["autor_img_src"] = header.find("img").get("src")
# print(autor_subtitle)

print(scrap)

{'autor_name': 'Marciana Agostinho', 'autor_subtitle': 'FrontEnd Software Engineer | React | React Native | Typescript | Jest | Next.js | Node.js | Firebase | Redux | Axios | Git | Github', 'post_age': '4d', 'autor_img_src': 'https://media.licdn.com/dms/image/D4D03AQGip6RnyQ76jQ/profile-displayphoto-shrink_400_400/0/1677454722050?e=2147483647&v=beta&t=ekahRf6_s5rDRW5htkja-vxaXCxLjAGgjSrw1sScoM0'}


In [7]:
scrap["content_text"] = soup_article.find("p", class_="attributed-text-segment-list__content").text
scrap["content_text"]

'Bom Dia! Rede.\n\nIniciar os estudos de SQL é como abrir uma porta para um mundo de descobertas emocionantes. É a emoção de dar os primeiros passos em direção ao entendimento e domínio de uma linguagem que permeia os bastidores de sistemas e aplicativos, guiando o fluxo de dados de forma precisa e eficiente. Cada linha de código escrita é uma oportunidade de compreender como estruturar consultas, manipular tabelas e extrair informações valiosas de conjuntos de dados aparentemente complexos. A satisfação em iniciar os estudos de SQL reside na sensação de capacitação e no reconhecimento de que estou adquirindo habilidades essenciais para enfrentar desafios cada vez mais relevantes no mundo da tecnologia e da análise de dados. É o primeiro passo em uma jornada fascinante de aprendizado e crescimento profissional.\n\n#Analytics #SQL #Analisededados #Conhecimento'

In [8]:
scrap["content_imgs_src"] = []

try:
    article_element = driver.find_element(by=By.TAG_NAME, value="article")

    soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")
    content_imgs = soup_article.find(attrs={"data-test-id": "feed-images-content"})
    if content_imgs:
        content_imgs = content_imgs.find_all("img")
        print("Post contem imagens")
        scrap["content_imgs_src"] = [img.get("src") for img in content_imgs]

    content_video = soup_article.find_all("video")
    if content_video:
        print("Contem video")
        scrap["content_imgs_src"] = [video.get("data-poster-url") for video in content_video]


except NoSuchElementException:
    print("Tipo de postagem não suportado. Falha ao capturar conteúdo")

if len(scrap["content_imgs_src"]) == 0:
    print("Não foi encontrado imagem ou videos no post")
    
print(scrap["content_imgs_src"])

Post contem imagens
['https://media.licdn.com/dms/image/D4E22AQG6qxpPIrfW7A/feedshare-shrink_2048_1536/0/1713222533151?e=2147483647&v=beta&t=6t-eXU28qWcAl2XRGbmy195S5GNFBw9Y4jngdu8c9fg']


In [9]:
#reactions
reactions = soup_article.find("div", class_= "main-feed-activity-card__social-actions").text

scrap["reactions"] = [item.strip() for item in reactions.split("\n") if item.strip()]
print(scrap["reactions"])



['129', '6 Comments']


In [10]:
# comentários
scrap["comments"] = []
article_element = driver.find_element(by=By.TAG_NAME, value="article")
soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")


comments = soup_article.find_all("section", class_="comment")

for comment in comments:
    current_comment = {}

    comment_header = comment.find(class_="comment__header").text.split("\n")
    comment_header_texts = [item.strip() for item in comment_header if item.strip()]
    current_comment["autor"] = comment_header_texts[0]
    current_comment["subtitle"] = comment_header_texts[1]
    current_comment["age"] = comment_header_texts[2]

    current_comment["profile_url"] = comment.find("a").get("href").split("?")[0]
    current_comment["profile_image_src"] = comment.find("img").get("src")
    current_comment["comment_text"] = comment.find(class_="comment__text").text
    scrap["comments"].append(current_comment)

# post

In [11]:
def clear_text(text):
    # Remove caracteres não permitidos e converte para minúsculas
    text = re.sub(r'[^a-zA-Z0-9]', '_', text.lower())
    return text


today = datetime.now().strftime("%Y-%m-%d")
folder_name_autor = clear_text(scrap["autor_name"])
folder_name_post = clear_text(scrap["content_text"][:20])

export_path = f"scraped/{today}/{folder_name_autor}/{folder_name_post}"

if not os.path.exists(export_path):
    os.makedirs(export_path)

with open(f"{export_path}/data.json", "w", encoding="utf-8") as file:
    json.dump(scrap, file, ensure_ascii=False)

In [12]:
for comment in scrap["comments"]:
    print(comment)

{'autor': 'Caroline Dutka', 'subtitle': 'Departamento Pessoal | Recursos Humanos', 'age': '4d', 'profile_url': 'https://br.linkedin.com/in/caroline-dutka-22a9ab2a3', 'profile_image_src': 'https://media.licdn.com/dms/image/D4D03AQE1keNrn3w7GA/profile-displayphoto-shrink_400_400/0/1707163091254?e=2147483647&v=beta&t=FJhqngWZ9pUH2oAEfbHAtXLxlAtgalxhIfjyGLF-vFs', 'comment_text': 'Esse mundo de descobertas é incrível. Parabéns pela dedicação'}
{'autor': 'Alex Rangel', 'subtitle': 'Desenvolvedor Full Stack | Front End | Back End', 'age': '4d', 'profile_url': 'https://br.linkedin.com/in/alex-rangel90', 'profile_image_src': 'https://media.licdn.com/dms/image/D4D03AQF6UGrIbXTH3A/profile-displayphoto-shrink_400_400/0/1713142385368?e=2147483647&v=beta&t=JOVihWJ9xHJLCIziHJIHyuH4IXieroTUojIQW1gzyII', 'comment_text': 'Parabéns pela dedicação '}
{'autor': 'Antonio Carlos Gonçalves Correa', 'subtitle': 'Analista de Dados | Power BI | Banco de Dados | Análise de Dados | Dashboards', 'age': '4d', 'profi

In [13]:
driver.get(scrap["autor_img_src"])
sleep(.5)
driver.find_element(by=By.TAG_NAME, value="img").screenshot(f"{export_path}/autor_img.png")

for index, img in enumerate(scrap["content_imgs_src"]):
    sleep(.5)
    driver.get(img)
    driver.find_element(by=By.TAG_NAME, value="img").screenshot(f"{export_path}/content_img_{index}.png")

if scrap["comments"]:
    for index, comment in enumerate(scrap["comments"]):
        sleep(.5)
        driver.get(comment["profile_image_src"])

        image_output_path = f"{export_path}/comment_profile_photo_{index}.png"

        comment["image_index"] = str(index)
        driver.find_element(by=By.TAG_NAME, value="img").screenshot(image_output_path)
        

In [14]:
driver.close()

In [15]:
import shutil
#copy to folder
input_folder = export_path
output_folder = "last_scrap"

#delete old folder
shutil.rmtree(output_folder, ignore_errors=True)

shutil.copytree(input_folder, output_folder)

'last_scrap'